In [1]:
import pandas as pd
import numpy as np
import json
import os
import sys
import pickle
from tqdm.notebook import tqdm as tqdm_n
from nltk.tokenize import word_tokenize 
from sklearn.model_selection import train_test_split
import gc
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import tensorflow as tf

from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation, Dropout
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [2]:
data_path = 'data'

In [3]:
train_data = np.load(os.path.join(data_path, 'twitter_train_vectors.npy'), allow_pickle=True)
test_data = np.load(os.path.join(data_path, 'twitter_test_vectors.npy'), allow_pickle=True)
train_labels = np.load(os.path.join(data_path, 'twitter_train_labels.npy'), allow_pickle=True)
test_labels = np.load(os.path.join(data_path, 'twitter_test_labels.npy'), allow_pickle=True)

In [4]:
def average_on_window(data, label, size):
    new_data = []
    new_labels = []
    for i in tqdm_n(range(len(data))):
        sample = data[i]
        if len(sample) >= size:
            new_sample = []
            count = int(np.floor((len(sample) / size)))
            rest = int(len(sample) / size)
            for j in range(size-1):
                new_sample.append(np.array(sample[j*count:(j+1)*count].mean(axis=0)))
            if rest != 0:
                new_sample.append(np.array(sample[(size-1)*count:(size)*count+rest].mean(axis=0)))
            else:
                new_sample.append(np.array(sample[(size-1)*count:(size)*count].mean(axis=0)))
            new_data.append(np.array(new_sample))
            new_labels.append(label[i])
    return np.array(new_data), np.array(new_labels)

In [5]:
X_train, y_train = average_on_window(train_data, train_labels, 2)

Widget Javascript not detected.  It may not be installed or enabled properly.


In [6]:
X_test, y_test = average_on_window(test_data, test_labels, 2)

Widget Javascript not detected.  It may not be installed or enabled properly.


In [7]:
tf.random.set_seed(42)
np.random.seed(42)
model = Sequential()

model.add(LSTM(5, return_sequences = True, activation='selu'))
model.add(Dropout(0.2))
model.add(LSTM(200, return_sequences=True, activation='selu'))
model.add(Dropout(0.2))
model.add(LSTM(100, return_sequences=False, activation='selu'))
model.add(Dropout(0.2))
model.add(Dense(100, activation='selu'))
model.add(Dense(50, activation='selu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

In [8]:
callbacks = [EarlyStopping(monitor='val_acc', min_delta=0.001, patience=5)]

In [9]:
%%time
model.fit(X_train, y_train, epochs=50, validation_split=0.1, batch_size=1024, callbacks=callbacks)

Train on 107277 samples, validate on 11920 samples
Epoch 1/50
107277/107277 [==============================] - 2s 23us/step - loss: 0.5676 - acc: 0.6996 - val_loss: 0.5002 - val_acc: 0.7551
Epoch 2/50
107277/107277 [==============================] - 1s 11us/step - loss: 0.5010 - acc: 0.7558 - val_loss: 0.4832 - val_acc: 0.7628
Epoch 3/50
107277/107277 [==============================] - 1s 11us/step - loss: 0.4877 - acc: 0.7663 - val_loss: 0.4742 - val_acc: 0.7718
Epoch 4/50
107277/107277 [==============================] - 1s 11us/step - loss: 0.4789 - acc: 0.7708 - val_loss: 0.4678 - val_acc: 0.7764
Epoch 5/50
107277/107277 [==============================] - 1s 12us/step - loss: 0.4735 - acc: 0.7743 - val_loss: 0.4660 - val_acc: 0.7788
Epoch 6/50
107277/107277 [==============================] - 1s 12us/step - loss: 0.4705 - acc: 0.7762 - val_loss: 0.4659 - val_acc: 0.7782
Epoch 7/50
107277/107277 [==============================] - 1s 12us/step - loss: 0.4673 - acc: 0.7788 - val_loss: 0

In [10]:
model.evaluate(X_test, y_test)

39725/39725 [==============================] - 3s 88us/step


[0.4573823452235919, 0.7847954630851746]

Usually they get an accuracy of about 83% on this dataset. I am using only a tenth of the original size and I achieved 78% with an LSTM network. This could be better with some more tuning but in this project we're focusing on our batch selection algorithm.